## Setup

In [4]:
# Uncomment if you need to install 
# !pip install random_survival_forest

In [2]:
import pandas as pd
import numpy as np
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
from random_survival_forest import RandomSurvivalForest, concordance_index

np.random.seed(1337)

In [3]:
D = sio.loadmat('data.mat')
#dict_keys(['__header__', '__version__', '__globals__', 'Censored', 'Integ_Symb_Types', 'Integ_Symbs', 'Integ_X', 'Integ_X_raw', 'Patients', 'Subtypes', 'Survival'])

T = np.asarray([t[0] for t in D['Survival']]).astype('float32')
O = 1 - np.asarray([c[0] for c in D['Censored']]).astype('int32')
X = D['Integ_X_raw'].astype('float32')
X_headers = list(D['Integ_Symbs'])

df = pd.DataFrame(data=X, columns=X_headers)
df['Survival'] = T
df['Censored'] = O
print(df.shape)
df.head()

(560, 401)


age_at_initial_pathologic_diagnosis_Clinical                   ...  Censored
0                                               50.0              ...         1
1                                               57.0              ...         1
2                                               53.0              ...         0
3                                               86.0              ...         1
4                                               66.0              ...         1

[5 rows x 401 columns]

In [4]:
y = df.iloc[:,-2:]
y["Survival"] = y["Survival"].values.astype(int)
X = df.iloc[:,:-2]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)
print("shape of X Train :"+str(X_train.shape))
print("shape of X Test :"+str(X_test.shape))
print("shape of Y Train :"+str(y_train.shape))
print("shape of Y Test :"+str(y_test.shape))

shape of X Train :(420, 399)
shape of X Test :(140, 399)
shape of Y Train :(420, 2)
shape of Y Test :(140, 2)


## Establish a baseline using Logistic Regression


In [ ]:
#logistic Regression

#logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train["Survival"])
y_pred=logreg.predict(X_test)
#cnf_matrix = metrics.confusion_matrix(y_test["Survival"], y_pred["Censored"])


#c_index = concordance_index(logreg, y_test, y_pred)
#print('C-index: {:.2f}'.format(c_index))
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
#print("Precision:",metrics.precision_score(y_test, y_pred))
#print("Recall:",metrics.recall_score(y_test, y_pred))
c_val = concordance_index(y_time=y_test["Survival"], y_pred=y_pred, y_event=y_test["Censored"])
print('C-index: {:.2f}'.format(c_val))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


C-index: 0.38


## Establish a baseline using Support Vector Machine (SVM)

In [ ]:
# Uncomment if you need to install 
#!pip install pysurvival

In [5]:
from pysurvival.models.svm import LinearSVMModel
from pysurvival.utils.metrics import concordance_index as pysurvival_concordance_index

In [38]:
svm_model = LinearSVMModel()
svm_model.fit(X=X_train, T=y_train["Survival"], E=y_train["Censored"], init_method='he_normal', lr = 0.5,  
              tol = 1e-3,  l2_reg = 1e-3, verbose = False)


LinearSVMModel

In [39]:
c_index = pysurvival_concordance_index(svm_model, X=X_test, T=y_test["Survival"], E=y_test["Censored"])
print('C-index: {:.2f}'.format(c_index))

C-index: 0.66


## Establish a baseline using Random Survival Forest (RSF)

In [10]:
from pysurvival.models.survival_forest import RandomSurvivalForestModel

rsf = RandomSurvivalForestModel(num_trees=20)
rsf.fit(X_train, y_train["Survival"], y_train["Censored"])
print(f'c-index of baseline RSF model: {pysurvival_concordance_index(rsf, X_test, y_test["Survival"], y_test["Censored"])}')

c-index of baseline RSF model: 0.7867007742861026


## Neural Network SurvivalNet Reimplementation

In [15]:
# Uncomment if you need to install 
#!pip install keras_tuner

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
import keras_tuner as kt

tf.random.set_seed(1337)

2021-08-12 13:43:35.473881: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-12 13:43:35.474091: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [13]:
class partial_log_likelihood(keras.losses.Loss):
    def __init__(self, name="partial_log_likelihood"):
        super().__init__(name=name)
    def call(self, y_true, y_pred):
        censored = y_true[:,1]
        y_true = y_true[:,0]
        y_pred = tf.reshape(y_pred, [-1])
        sorted_survival, sorted_indices = tf.math.top_k(y_true, len(y_true), True) 
        sorted_censored = keras.backend.gather(censored, sorted_indices)
        sorted_preds = keras.backend.gather(y_pred, sorted_indices)
        exp = keras.backend.exp(sorted_preds)
        sums = keras.backend.cumsum(exp) + tf.convert_to_tensor(1.0)
        logsums = keras.backend.log(sums)
        
        return keras.backend.sum(sorted_censored * sorted_preds - sorted_censored*logsums) * -1

In [14]:
# Model outline with hyperparameter ranges/choices to be searched by BayesOpt tuner
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int("units1", min_value=32, max_value=512, step=32), activation='relu', input_dim=len(X_train.columns)))
    model.add(Dropout(hp.Float("drop_rate", min_value=0.0, max_value=0.2, step=0.1)))
    model.add(Dense(hp.Int("units2", min_value=32, max_value=512, step=32), activation='relu'))
    model.add(Dropout(hp.Float("drop_rate", min_value=0.0, max_value=0.2, step=0.1)))
    model.add(Dense(hp.Int("units3", min_value=32, max_value=512, step=32), activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])),loss=partial_log_likelihood(), run_eagerly=True)
    return model

In [15]:
# Run Bayesian Optimization on the NN survival model to find best hyperparameters
tuner = kt.BayesianOptimization(build_model, objective="loss", max_trials=20, executions_per_trial=1, directory="./bayes_opt", project_name="logs")
y_train.loc[:,"Censored"] = np.array(y_train.loc[:,"Censored"].values).astype(np.float32)
tuner.search(X_train, y_train.values, batch_size=32, epochs=50)

INFO:tensorflow:Reloading Oracle from existing project ./bayes_opt/logs/oracle.json


2021-08-12 13:44:47.770181: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-12 13:44:47.770431: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-12 13:44:47.770557: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-KHDBI9N): /proc/driver/nvidia/version does not exist
2021-08-12 13:44:47.794600: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Reloading Tuner from ./bayes_opt/logs/tuner0.json
INFO:tensorflow:Oracle triggered exit


/home/jjcc/.cache/pypoetry/virtualenvs/survivalnet-reimplementation-nhtVa1r6-py3.8/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [16]:
# Rebuild model with best hyperparameters selected by bayesian optimization
best_hp = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hp)

In [17]:
# Retrain best model
model.fit(X_train, y_train.values,batch_size=32, epochs=50, verbose=False)

2021-08-12 13:45:00.816819: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-12 13:45:00.829701: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2399995000 Hz


In [18]:
print(f'c-index: {concordance_index(y_time=y_test["Survival"], y_pred=model.predict(X_test), y_event=y_test["Censored"])}')

c-index: 0.8597711811997526
